In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

In [2]:
linkList = []

driver_path = "msedgedriver.exe"
service = Service("msedgedriver.exe")
driver =  webdriver.Edge(service=service)

driver.get("https://www.watsons.com.tr/stoklari-bitiriyoruz/c/50021?q=%3A%3AinStockFlag%3AStokta%2BOlmayanlar")  

links = driver.find_elements(By.CSS_SELECTOR, "#products-content-list a")

hrefs = [link.get_attribute("href") for link in links]

for href in hrefs:
    linkList.append(href)

driver.quit()

In [ ]:
counter = 1
while counter<88:

    driver.get("https://www.watsons.com.tr/stoklari-bitiriyoruz/c/50021?q=%3A%3AinStockFlag%3AStokta%2BOlmayanlar&page="+str(counter))  

    links = driver.find_elements(By.CSS_SELECTOR, "#products-content-list a")

    hrefs = [link.get_attribute("href") for link in links]

    for href in hrefs:
        linkList.append(href)

    counter = counter + 1
    time.sleep(5)
driver.quit()


In [3]:
# Ürün linki haricinde alınan linkleri temizlemek için bir kontrol yazıyoruz.
linkList2 = []
for i in range(len(linkList)):
    if "page" not in linkList[i] and "stoklari" not in linkList[i]:
        linkList.append(linkList[i])

In [4]:
# Birden fazla kaydedilen linkleri temizlemek için pandasın drop_duplicates fonksiyonunu kullanıyoruz.
linkData = pd.DataFrame(linkList)
droppedData = linkData.drop_duplicates()

In [5]:
# DataFrame'in indekslerini sıralı indekslerle değiştiriyoruz.
droppedData = droppedData.rename(index={j: i for i, j in enumerate(droppedData.index)})

In [ ]:
stoktakiUrunler = []
service = Service("msedgedriver.exe")
driver =  webdriver.Edge(service=service)
i = 0
while i < (len(droppedData)):
        
    sayfaLink = droppedData[0][i]
    driver.get(sayfaLink)
    time.sleep(2)

    if(i == 0):
        driver.find_element(By.ID, "onetrust-accept-btn-handler").click()

    try:
        driver.find_element(By.CLASS_NAME, "rts-cta").click()
        wait = WebDriverWait(driver, 10)
        aramaKutusu = wait.until(EC.visibility_of_element_located((By.ID, "search")))
        # Görünür hale geldikten sonra metni gir
        aramaKutusu.send_keys("İstanbul")

        driver.find_element(By.CLASS_NAME, "store-locator-search-bar__submit").click()

        time.sleep(2)

        try:
            errorMessage = driver.find_element(By.CLASS_NAME, "store-locator-search-bar__error").text

        except NoSuchElementException:
            print(sayfaLink)
            stoktakiUrunler.append(sayfaLink)

        time.sleep(5)

    except NoSuchElementException:
        time.sleep(5)

    i = i + 1        

driver.quit()